In [30]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [31]:
df = pd.read_csv("../datasets/titanic/train.csv")

In [32]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
data = df.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)

In [34]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [35]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch
count,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594
std,0.486592,0.836071,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000


In [36]:
le = LabelEncoder()
data["Sex"] = le.fit_transform(data.Sex)


In [37]:
le.classes_

array(['female', 'male'], dtype=object)

In [39]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0


In [40]:
data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000


In [41]:
complete = data.dropna()

In [49]:
table = complete.groupby(["Sex", "Pclass"]).mean()[["Age"]].reset_index()

In [50]:
table

,Sex,Pclass,Age
0,0,1,34.611765
1,0,2,28.722973
2,0,3,21.750000
3,1,1,41.281386
4,1,2,30.740707
5,1,3,26.507589


In [74]:
def fill_age(row):
    if row.isna()["Age"]:
        row["Age"] = table.loc[(table.Sex == row["Sex"]) & (table.Pclass == row["Pclass"])].Age.mean()
    
    return row

In [76]:
processed = data.apply(fill_age, axis=1)

In [78]:
processed.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0.0,3.0,1.0,22.0,1.0,0.0
1,1.0,1.0,0.0,38.0,1.0,0.0
2,1.0,3.0,0.0,26.0,0.0,0.0
3,1.0,1.0,0.0,35.0,1.0,0.0
4,0.0,3.0,1.0,35.0,0.0,0.0


In [68]:
# from sklearn.tree import DecisionTreeRegressor

# model = DecisionTreeRegressor()

# model.fit(complete[["Sex", "Pclass"]], complete.Age)

# model.predict([[1, 1]])

In [71]:
table.loc[(table.Sex == 0) & (table.Pclass == 1)].Age.mean()

34.61176470588235

In [79]:
X, y = processed.drop(["Survived"], axis=1), processed.Survived

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [88]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=5)

In [89]:
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [90]:
model.score(X_test, y_test)

0.8338983050847457

In [87]:
processed.to_csv("processed.csv", index=False)